In [ ]:
from PIL import Image
import urllib.request
from io import BytesIO
import re
import torch

image_url = 'https://raw.githubusercontent.com/katanaml/sparrow/main/sparrow-data/docs/input/invoices/processed/images/invoice_10.jpg'

with urllib.request.urlopen(image_url) as url:
    image = Image.open(BytesIO(url.read()))

# image.show()

In [ ]:
from huggingface_hub import login

login("")

In [ ]:
import re
import transformers
from PIL import Image
from transformers import DonutProcessor, VisionEncoderDecoderModel
import torch
import random
import numpy as np

# hidde logs
transformers.logging.disable_default_handler()


# Load our model from Hugging Face
processor = DonutProcessor.from_pretrained("katanaml-org/invoices-donut-model-v2")
model = VisionEncoderDecoderModel.from_pretrained("katanaml-org/invoices-donut-model-v2")

# Move model to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

In [ ]:
def run_prediction(sample, model=model, processor=processor):
    # prepare encoder inputs
    pixel_values = processor(image, return_tensors="pt").pixel_values

    # prepare decoder inputs
    task_prompt = "<s>"
    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

    # run inference
    outputs = model.generate(
        pixel_values.to(device),
        decoder_input_ids=decoder_input_ids.to(device),
        max_length=model.decoder.config.max_position_embeddings,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        num_beams=1,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )

    # process output
    prediction = processor.batch_decode(outputs.sequences)[0]
    prediction = processor.token2json(prediction)

    return prediction

prediction = run_prediction(image)
print(f"Prediction:\n {prediction}")